In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

print(os.listdir("../input"))
ipath="../input/plant-pathology-2020-fgvc7/images/"
sam_path="../input/plant-pathology-2020-fgvc7/sample_submission.csv"
sam_data=pd.read_csv(sam_path)
train_data=pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")
test_data=pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")
len(sam_data)
len(train_data)
len(test_data)

In [ ]:
train_data.head(5)
print(sam_data)

In [ ]:
path=ipath+train_data.image_id[130]+'.jpg'

In [ ]:
img=plt.imread(path)
print(img.shape)
plt.imshow(img)

In [ ]:
from PIL import Image
imgsize=(210,210)
def load_images(dir_path,csv_data):
    d=[]
    for i in csv_data.image_id:
        img=Image.open(dir_path+i+'.jpg')
        img=img.resize(imgsize,Image.NEAREST)
        d.append(np.asarray(img)/255)
    d=np.array(d)
    return d,csv_data[csv_data.columns[1:]]


In [ ]:
data,labels=load_images(ipath,train_data)

In [ ]:
data.shape

In [ ]:
labels.shape

In [ ]:
from keras.regularizers import l2
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,Dropout
from keras.models import Sequential
rp=0.0001
model=Sequential()
model.add(Conv2D(12,(3,3),activation='relu',padding='same',kernel_regularizer=l2(rp),input_shape=(imgsize[0],imgsize[1],3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='tanh',padding='same',kernel_regularizer=l2(rp)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='tanh',padding='same',kernel_regularizer=l2(rp)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu',padding='same',kernel_regularizer=l2(rp)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu',padding='same',kernel_regularizer=l2(rp)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='tanh',padding='same',kernel_regularizer=l2(rp)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu',padding='same',kernel_regularizer=l2(rp)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout((0.2)))
model.add(Flatten())
model.add(Dense(4,activation='sigmoid'))


In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(data,labels,batch_size=20,epochs=5)

In [ ]:
model_json=model.to_json()
with open('model_json',"w") as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')
print("saved model to disk")

In [ ]:
t_data,t_labels=load_images(ipath,test_data)

In [ ]:
t_data.shape

In [ ]:
result=model.predict_proba(t_data)
result

In [ ]:
np.set_printoptions(formatter={'float':lambda x:"{0:0.2f}".format(x)})

In [ ]:
sam_data[sam_data.columns[1:]]=pd.DataFrame(data=result)

In [ ]:
sam_data

In [ ]:
sam_data.to_csv("yresult3.csv")